In [13]:
from dataclasses import dataclass
from functools import partial

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

from run_everything import compute_spectra, VariableArg

In [14]:
# Static args

COMMON_ARGS = {
    "temperature_kelvin": 300,
    "mode_frequencies": [1700, 100],
    # "mode_couplings": [1.5, 6.67],
}

TWO_STATE_ARGS = {
    "transfer_integral": 100,
    "broadening": 200,
    "mode_basis_sets": [20, 200],
}

MLJ_ARGS = {
    "disorder_meV": 0,
    "basis_size": 20,
}

STARK_ARGS = {
    "positive_field_sum_percent": 0.5,
}

In [15]:
# Variable args (modified for sliders)

@dataclass
class VariableSliderArg(VariableArg):
    min: float
    max: float
    default: float

    def __init__(self, values, default=None):
        self.values = np.array(values)
        self.min = np.min(self.values)
        self.max = np.max(self.values)
        
        if default is None:
            default = self.values[0]
        self.default = default

    @classmethod
    def from_minmax(cls, min, max, num_points, default=None):
        cls_initializer = partial(cls, default=default)
        return cls_initializer(np.linspace(min, max, num_points))
    
    def to_slider(self, name):
        return widgets.SelectionSlider(
            options=self.values,
            value=self.default,
            description=name,
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True
        )


VARIABLE_ARGS = {
    "energy_gap": VariableSliderArg([0, 200, 400, 800, 8_000]),
    "g1": VariableSliderArg([0.7]),
    "g2": VariableSliderArg.from_minmax(2, 8, 4),
    "positive_field_strength": VariableSliderArg([0.0, 0.01, 0.1]),
    "field_delta_dipole": VariableSliderArg.from_minmax(0, 38, 3),
    "field_delta_polarizability": VariableSliderArg.from_minmax(0, 1_000, 3),
}


In [17]:
# Plotting (abs against experiment?)
from experiment.sample_experiment import x as x_expr
from experiment.sample_experiment import y_smooth as y_expr_smooth

def plot(**variable_args):
    plt.clf()
    
    lower_bound = variable_args.pop("lower bound")
    
    all_args = {
        **COMMON_ARGS,
        **variable_args,
        **TWO_STATE_ARGS,
        **MLJ_ARGS,
        **STARK_ARGS,
    }
    
    if "g1" and "g2" in all_args:
        g1 = all_args.pop("g1")
        g2 = all_args.pop("g2")
        all_args["mode_couplings"] = [g1, g2]
    
    _, _, two_state_stark_spectrum, _ = compute_spectra(all_args, force_loading=True)
    
    two_state_stark_spectrum = two_state_stark_spectrum.cut_bounds(start_energy=lower_bound)
    
    plt.plot(two_state_stark_spectrum.energies, two_state_stark_spectrum.intensities)
    
    # y_expr = y_expr_smooth * np.max(two_state_spectrum.intensities) / np.max(y_expr_smooth)
    # plt.plot(x_expr, y_expr, label="experiment")
    
# add bounds to variable args
variable_args = {
    **VARIABLE_ARGS,
    "lower bound": VariableSliderArg.from_minmax(0, 4000, num_points=20),
}
    
widgets.interact(plot, **{k: v.to_slider(k) for k, v in variable_args.items()})

interactive(children=(SelectionSlider(continuous_update=False, description='energy_gap', options=(0, 200, 400,…

<function __main__.plot(**variable_args)>